In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def make_doc2vec_models(tagged_data, tok, vector_size=128, window = 3, epochs = 40, min_count = 0, workers = 4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'./datas/{tok}_news_model.doc2vec')

In [15]:
def get_data(preprocess = True):
    if preprocess :
        data = pd.read_csv('./datas/naver_news/preprocessing_tok_naver_data.csv')
    else:
        economy = pd.read_csv('./datas/naver_news/economy.csv', header=None)
        policy = pd.read_csv('./datas/naver_news/policy.csv', header=None)
        it = pd.read_csv('./datas/naver_news/it.csv', header=None)

        columns = ['date', 'category', 'company', 'title', 'content', 'url']
        economy.columns = columns
        policy.columns = columns
        it.columns = columns

        data = pd.concat([economy, policy, it], axis = 0)
        data.reset_index(drop=True, inplace=True)
    
    return data

In [16]:
def get_preprocessing_data(data):
    data.drop(['date', 'company', 'url'], axis = 1, inplace =True)
    
    category_mapping = {
    '경제' : 0,
    '정치' : 1,
    'IT과학' : 2
    }

    data['category'] = data['category'].map(category_mapping)
    data['title_content'] = data['title'] + " " + data['content']
    data.drop(['title', 'content'], axis = 1, inplace = True)
    return data

In [17]:
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

In [18]:
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:5]
    
    return data.loc[scores, :]

In [19]:
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

In [20]:
def view_user_history(data):
    print(data[['category', 'title_content']])

In [21]:
data = get_data()

In [22]:
data.head()

,category,title_content,mecab_tok
0,0,포스코ICT 4분기부터 실적 본격 개선 비용효율화 본격화 그룹 계열사와의 시너지 발...,포스코 ICT 분기 실 본격 개선 비용 효율 본격화 그룹 계열사 시너지 발생 헤럴드...
1,0,위메프 리퍼데이 바자회 수익금 소외계층 지원 위메프가 리퍼데이 바자회 ‘아름다운가게...,위메프 리퍼 데이 바자회 수익금 소외 계층 지원 위메프 리퍼 데이 바자회 가게 위메...
2,0,호반건설 광주시 ‘계림1구역 정비사업’ 수주…공사비 2700억원 최고 33층 총 9...,호반건설 광주시 계림 구역 정비 사업 수주 공사비 원 최고 개동 아파트 가구 규모 ...
3,0,성동조선해양 HSG중공업과 인수합병 MOU 체결 HSG중공업이 성동조선해양 인수를 ...,성동 조선 해양 중공업 인수 합병 체결 중공업 성동 조선 해양 인수 위한 양해 각서...
4,0,2019년 10월 프랜차이즈 정보공개서 79개 신규등록 서울·경기·인천 지역 서울시...,년 월 프랜차이즈 정보 공개 개 신규 등록 서울 경기 인천 지역 서울시 경기도 새롭...


In [23]:
data_doc_title_content_tag = make_doc2vec_data(data, 'title_content', t_document=True)
data_doc_title_content = make_doc2vec_data(data, 'title_content')
data_doc_tok_tag = make_doc2vec_data(data, 'mecab_tok', t_document=True)
data_doc_tok = make_doc2vec_data(data, 'mecab_tok')

# make doc2vec models

In [24]:
make_doc2vec_models(data_doc_title_content_tag, tok=False)

In [25]:
make_doc2vec_models(data_doc_tok_tag, tok=True)

# load doc2vec models

In [26]:
model_title_content = Doc2Vec.load('./datas/False_news_model.doc2vec')

In [27]:
model_tok = Doc2Vec.load('./datas/True_news_model.doc2vec')

In [49]:
user_category_1 = data.loc[random.sample(data.loc[data.category == 0, :].index.values.tolist(), 5), :]  #경제
view_user_history(user_category_1)

      category                                      title_content
510          0  SK 조직 차원 증거인멸 없어 반박문 제출 LG화학과 영업비밀 침해소송을 벌이고 있...
4187         0  제이엘케이인스펙션 공모가 9000원 확정 김동민 대표 적극적인 IR·주주친화 정책 ...
213          0  제재 대신 자율개선 유도하는 금감원 외환법규 위반 5개 은행 대상 첫 적용 금융감독...
696          0  동국제약 ‘마데카솔’ 소비자가 가장 추천하는 브랜드 동국제약 대표이사 오흥주 마데카...
3410         0  호반그룹 정기 인사…최승남 대표 총괄부회장 신규 선임 한국경제TV 신인규 기자 호반...


In [45]:
user_category_2 = data.loc[random.sample(data.loc[data.category == 1, :].index.values.tolist(), 5), :]  #정치
view_user_history(user_category_2)

       category                                      title_content
10671         1  “3대 친문 농단” 국조 요구서 제출… 靑 매섭게 몰아치는 야권 자유한국당 곽상도ㆍ...
6194          1  黃 다시 일어나 끝까지 가겠다… 공수처·선거법 반드시 저지 黃 文정부 3대 게이트 ...
6825          1  미 대사 文대통령 겨냥 종북 좌파에 둘러싸여... 9월 여야 의원들 만나 위험 발언...
8533          1  명소·맛집 즐긴 아세안 정상…일정보다 핫했던 부산 나들이 2박 3일간 경찰 에스코트...
6236          1  민주 한국당 필리버스터 철회 없으면 정기국회서 41로 안건 처리 비공개 최고위원회…...


In [18]:
user_category_3 = data.loc[random.sample(data.loc[data.category == 2, :].index.values.tolist(), 5), :]  #IT 과학
view_user_history(user_category_3)

       category                                      title_content
14551         2  ‘라인프렌즈’ ‘브롤스타즈’와 함께 ‘브라운앤프렌즈’ 팝업스토어 진행 엑스포츠뉴스닷...
17055         2  올해 1등 KT인상 대상 ‘5G 경쟁력 강화 TF’ 디지털데일리 최민지기자 KT 대...
17824         2  DID 시장 열린다 2.은행계좌부터 주식까지 블록체인으로 인증 OK 규제 샌드박스 ...
12625         2  인하대병원 제로페이 도입 한국간편결제진흥원 이사장 윤완수 은 인하대병원에서 제로페이...
18243         2  이슈 19금 신작 미소녀 RPG 방치소녀 학원편 신규 캐릭터 조운과 문추 추가 본 ...


In [50]:
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_title_content, model_title_content) # 경제
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_title_content, model_title_content) # 정치
user_3 = make_user_embedding(user_category_3.index.values.tolist(), data_doc_title_content, model_title_content) # IT과학

In [51]:
result = get_recommened_contents(user_1, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
5226,0,삼양그룹 정기 임원인사 실시…“성과주의 실현” 승진 10명 보직변경 5명 김지섭 삼...
3933,0,동국제약 바이오의약품 위탁 개발·생산 사업 진출 서울 연합뉴스 동국제약은 지난 달 ...
5251,0,삼양그룹 정기임원인사 단행...김지섭 부사장 승진 왼쪽부터 삼양사 식자재유통BU장 ...
3329,0,호반그룹 총괄부회장에 최승남 대표 선임 2일 정기 임원인사 단행.. 각 계열사 대표...
13015,2,동국제약 바이오의약품 위탁 개발·생산 사업 진출 서울 연합뉴스 동국제약은 지난 달 ...


In [48]:
result = get_recommened_contents(user_2, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
8228,1,노영민 비서실장과 정의용 국가안보실장 서울 연합뉴스 한상균 기자 청와대 노영민 비서...
6348,1,한국당 친문농단 게이트 국정조사 요구서 제출키로 울산에 백원우팀 파견…靑 고래고기 ...
6320,1,한국당 친문농단 게이트 국정조사 요구서 제출하기로 자유한국당 곽상도 의원이 28일 ...
6554,1,박원순 시장 녹색교통지역 5등급차량 운행제한 상황실 방문 서울 뉴시스 박주성 기자 ...
6842,1,단식 종료 황교안 내일부터 한국당 당무 복귀 서울 연합뉴스 홍정규 기자 단식농성을 ...


In [22]:
result = get_recommened_contents(user_3, data_doc_title_content, model_title_content)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
13073,2,브롤스타즈 라인프렌즈 오리지널 캐릭터 등장 슈퍼셀은 라인프렌즈와 모바일 슈팅 게임 ...
16573,2,넥슨 글로벌 멀티 플랫폼 프로젝트 ‘카트라이더 드리프트’ 테스트 넥슨 대표 이정헌 ...
11608,1,사진한국당 규탄하는 야3당 대표 머니투데이 홍봉진 기자 바른미래당 손학규 정의당 심...
16536,2,카트라이더 드리프트 9일까지 비공개 테스트 진행 넥슨은 멀티 플랫폼 프로젝트 카트라...
13387,2,슈퍼셀 라인프렌즈와 ‘브롤스타즈’ IP 라이선싱 계약 브롤스타즈 캐릭터 상품 개발 ...


# 형태소 분석 후 결과

In [52]:
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_tok, model_tok) # 경제
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_tok, model_tok) # 정치
user_3 = make_user_embedding(user_category_3.index.values.tolist(), data_doc_tok, model_tok) # IT과학

In [53]:
result = get_recommened_contents(user_1, data_doc_tok, model_tok)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
11226,1,개회사 하는 조성욱 공정거래위원장 서울 연합뉴스 임헌정 기자 조성욱 공정거래위원장이...
11702,1,국민의례하는 문재인 대통령 문재인 대통령이 3일 청와대에서 열린 국가기후환경회의 격...
4683,0,정관장 알파프로젝트와 상담하세요 서울 뉴시스 박주성 기자 KGC인삼공사가 2일 오전...
6391,1,미래를 향한 전진 4.0 창당발기인대회에서 박수치는 이언주 서울 연합뉴스 하사헌 기...
11325,1,국민의례 하는 이낙연 총리 서울 뉴스1 오대일 기자 이낙연 국무총리가 3일 오후 서...


In [54]:
result = get_recommened_contents(user_2, data_doc_tok, model_tok)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
4794,0,권태신 부회장 주한아르헨티나 대사 접견 서울 뉴시스 권태신 전경련 부회장이 2일 서...
8434,1,육해공 철통 방어 한·아세안 모터케이드 행렬 부산 연합뉴스 지난달 부산에서 한·아세...
733,0,부산항 수출입 화물 가득 부산 연합뉴스 조정호 기자 1일 부산항 신선대부두에 수출입...
4846,0,인사 한화토탈 한화토탈은 2일 임원 승진 인사를 발표했다. 승진자는 상무 1명 상무...
2908,0,인사말하는 김기문 회장 서울 뉴스1 허경 기자 김기문 중소기업중앙회장이 2일 서울 ...


In [55]:
result = get_recommened_contents(user_3, data_doc_tok, model_tok)
pd.DataFrame(result.loc[:, ['category', 'title_content']])

,category,title_content
10471,1,한국 부임한 도미타 고지 신임 주한 일본대사 서울 연합뉴스 임헌정 기자 도미타 고지...
10517,1,입국하는 도미타 고지 신임 주한일본대사 CBS노컷뉴스 이한형 기자 도미타 고지 신임...
11084,1,청와대 천막농성장 앞에서 열린 한국당 최고위원회의 서울 뉴스1 박세연 기자 3일 오...
8353,1,수보회의 참석하는 문 대통령 서울 연합뉴스 한상균 기자 문재인 대통령이 2일 오후 ...
16157,2,Tech BIZ AI로 블루·화이트칼라 넘어 뉴칼라 일자리 만들 것 데이비드 반스...


형태소 분석 후 결과는 성능이 썩 좋지 않음을 알 수 있다.